**Links**<br>
Extensions API GitHub: https://github.com/SillyTavern/SillyTavern-extras/<br>
SillyTavern community Discord (support and discussion): https://discord.gg/RZdyAEUPvj

In [ ]:
#@title <-- Tap this if you run on Mobile { display-mode: "form" }
#Taken from KoboldAI colab
%%html
<b>Press play on the audio player to keep the tab alive. (Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@markdown Enables hosting of extensions backend for SillyTavern Extras
use_cpu = False #@param {type:"boolean"}
#@markdown Allows to run SillyTavern Extras on CPU (use if you're out of daily GPU allowance)
use_sd_cpu = False #@param {type:"boolean"}
#@markdown Allows to run Stable Diffusion pipeline on CPU (slow!)
extras_enable_captioning = True #@param {type:"boolean"}
#@markdown Loads the image captioning module
Captions_Model = "Salesforce/blip-image-captioning-large" #@param [ "Salesforce/blip-image-captioning-large", "Salesforce/blip-image-captioning-base" ]
#@markdown * Salesforce/blip-image-captioning-large - good base model
#@markdown * Salesforce/blip-image-captioning-base - slightly faster but less accurate
extras_enable_emotions = True #@param {type:"boolean"}
#@markdown Loads the sentiment classification model
Emotions_Model = "nateraw/bert-base-uncased-emotion" #@param ["nateraw/bert-base-uncased-emotion", "joeddav/distilbert-base-uncased-go-emotions-student"]
#@markdown * nateraw/bert-base-uncased-emotion = 6 supported emotions<br>
#@markdown * joeddav/distilbert-base-uncased-go-emotions-student = 28 supported emotions
extras_enable_memory = True #@param {type:"boolean"}
#@markdown Loads the story summarization module
Memory_Model = "slauw87/bart_summarisation" #@param [ "slauw87/bart_summarisation", "Qiliang/bart-large-cnn-samsum-ChatGPT_v3", "Qiliang/bart-large-cnn-samsum-ElectrifAi_v10", "distilbart-xsum-12-3" ]
#@markdown * slauw87/bart_summarisation - general purpose summarization model
#@markdown * Qiliang/bart-large-cnn-samsum-ChatGPT_v3 - summarization model optimized for chats
#@markdown * Qiliang/bart-large-cnn-samsum-ElectrifAi_v10 - nice results so far, but still being evaluated
#@markdown * distilbart-xsum-12-3 - faster, but pretty basic alternative
extras_enable_silero_tts = True #@param {type:"boolean"}
#@markdown Enables Silero text-to-speech module
extras_enable_edge_tts = True #@param {type:"boolean"}
#@markdown Enables Microsoft Edge text-to-speech module
extras_enable_sd = True #@param {type:"boolean"}
#@markdown Enables SD picture generation
SD_Model = "ckpt/anything-v4.5-vae-swapped" #@param [ "ckpt/anything-v4.5-vae-swapped", "hakurei/waifu-diffusion", "philz1337/clarity", "prompthero/openjourney", "ckpt/sd15", "stabilityai/stable-diffusion-2-1-base" ]
#@markdown * ckpt/anything-v4.5-vae-swapped - anime style model
#@markdown * hakurei/waifu-diffusion - anime style model
#@markdown * philz1337/clarity - realistic style model
#@markdown * prompthero/openjourney - midjourney style model
#@markdown * ckpt/sd15 - base SD 1.5
#@markdown * stabilityai/stable-diffusion-2-1-base - base SD 2.1
extras_enable_chromadb = True #@param {type:"boolean"}
#@markdown Enables ChromaDB for Infinity Context plugin

import subprocess

# ---
# SillyTavern extras
extras_url = '(disabled)'
params = []
if use_cpu:
    params.append('--cpu')
if use_sd_cpu:
    params.append('--sd-cpu')
params.append('--share')
ExtrasModules = []

if (extras_enable_captioning):
  ExtrasModules.append('caption')
if (extras_enable_memory):
  ExtrasModules.append('summarize')
if (extras_enable_emotions):
  ExtrasModules.append('classify')
if (extras_enable_sd):
  ExtrasModules.append('sd')
if (extras_enable_silero_tts):
  ExtrasModules.append('silero-tts')
if extras_enable_edge_tts:
  ExtrasModules.append('edge-tts')
if (extras_enable_chromadb):
  ExtrasModules.append('chromadb')

params.append(f'--classification-model={Emotions_Model}')
params.append(f'--summarization-model={Memory_Model}')
params.append(f'--captioning-model={Captions_Model}')
params.append(f'--sd-model={SD_Model}')
params.append(f'--enable-modules={",".join(ExtrasModules)}')


%cd /
!git clone https://github.com/Cohee1207/SillyTavern-extras
%cd /SillyTavern-extras
!git clone https://github.com/Cohee1207/tts_samples
!npm install -g localtunnel
!pip install -r requirements-complete.txt
!pip install tensorflow==2.12
!wget https://github.com/cloudflare/cloudflared/releases/download/2023.5.0/cloudflared-linux-amd64 -O /tmp/cloudflared-linux-amd64
!chmod +x /tmp/cloudflared-linux-amd64


cmd = f"python server.py {' '.join(params)}"
print(cmd)
extras_process = subprocess.Popen(
    cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd='/SillyTavern-extras', shell=True)
print('processId:', extras_process.pid)
while True:
    line = extras_process.stdout.readline().decode().strip()
    if line != None and line != '':
        print(line)
